In [ ]:
!pip install hsfs[hive] -y

In [ ]:
import hsfs

# TODO: replace the values below: [UUID], [project-name], [api-key]
connection = hsfs.connection(host="[UUID].cloud.hopsworks.ai",
    project="[project-name]",
    engine="hive",
    api_key_value="[api-key]")

fs = connection.get_feature_store()

## Show the first 5 rows in a Feature Group
A feature group is a set of related `features`. A feature is a data point that helps make predictions. A feature data value (or point) is often either a number (scalar, vector, etc) or a boolean or enum or string (categorical value).  If you are a data engineer, think of features in feature groups as columns in a database. If you are a data scientist, think of features in feature groups as columns in a dataframe.

In [ ]:
teams_features = fs.get_feature_group("teams_features",version=1)
teams_features.show(5)

## Ingest some features into the Feature Store as a Feature Group
The date we will ingest looks as follows:

 * first_name : string (categorical value)
 * last_name : string (categorical value)
 * country : string (categorical value)
 
 We want to use these features later to predict the country a first_name,last_name pair come from.

In [ ]:
import pandas as pd
try:
    name_country_fg = fs.get_feature_group(name="name_country_fg",version=1)
except Exception as e: 
    url = "https://repo.hops.works/dev/jdowling/data_cleaned_train.csv"
    df = pd.read_csv(url, sep=";")
    name_country_fg = fs.create_feature_group(name="name_country_fg",
                                    version=1,
                                    primary_key=['first_name', 'last_name'],
                                    description="Name - Country prediction",
                                    validation_type="STRICT",
                                    time_travel_format="HUDI",
                                    online_enabled=True,                                        
                                    statistics_config=True)
    name_country_fg.save(df)

In [ ]:
print("Name: {}".format(name_country_fg.name))
print("Description: {}".format(name_country_fg.description))
print("Features:")
features = name_country_fg.features
for feature in features:
    print("{:<60} \t Primary: {} \t Partition: {}".format(feature.name, feature.primary, feature.partition))

## Feature Data Validation

Garbage in, garbage out.

Let's check for garbage in.

In [ ]:
from hsfs.rule import Rule
rules = connection.get_rules()
[print(rule.to_dict()) for rule in rules]

In [ ]:
expectation_countries = fs.create_expectation("countries",
                                          description="min and max number of countries",
                                          features=["country"], 
                                          rules=[Rule(name="HAS_NUMBER_OF_DISTINCT_VALUES", level="ERROR", min=1), 
                                                 Rule(name="HAS_NUMBER_OF_DISTINCT_VALUES", level="ERROR", max=195)])
expectation_countries.save()

In [ ]:
name_country_fg.attach_expectation(expectation_countries)

In [ ]:
# Create a Pandas Dataframe and ingest its features into a feature group that you create here.  
import pandas as pd 
columns = ['first_name', 'last_name', 'country']
data = [['tom', 'johnson', 'UK'], ['penelope', 'charles', 'UK'], ['harry', 'windsor', "USA"]]   
df = pd.DataFrame(data, columns=columns) 
name_country_fg.insert(df)

In [ ]:
exps = name_country_fg.get_expectations()
[print(exp.description) for exp in exps]

In [ ]:
fg_validations = name_country_fg.get_validations()
[print(validation.to_dict()) for validation in fg_validations]

In [ ]:
import string
import random
def id_generator(size=1500, chars=string.ascii_uppercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

num_rows = 600

data = np.array([id_generator() for i in range(num_rows)]).reshape(200,3)
df2 = pd.DataFrame(data, columns=columns)
name_country_fg.insert(df2)

In [ ]:
fg_validations = name_country_fg.get_validations()
[print(validation.to_dict()) for validation in fg_validations]